# Metadata collection with ZnTrack

ZnTrack allows for the collection of some metadata.
One example is measuring the execution time of Nodes or even methods inside the Nodes easily.
This can be achieved by using the `@TimeIt` decorator which is shown in the following example.

In [1]:
from zntrack import Node, dvc, config
from zntrack.metadata import TimeIt
from time import sleep

config.nb_name = "05_metadata.ipynb"

In [2]:
from zntrack.utils import cwd_temp_dir
temp_dir = cwd_temp_dir()

In [3]:
!git init
!dvc init

Initialized empty Git repository in /tmp/tmpdv7hyzt_/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [4]:
@Node(silent=True)
class SleepNode:
    @TimeIt
    def run(self):
        self.sleep_1s()
        self.sleep_2s()
    @TimeIt
    def sleep_1s(self):
        sleep(1)
    def sleep_2s(self):
        sleep(2)

In [5]:
SleepNode()()

In [6]:
!dvc repro

Running stage 'SleepNode':                                            core>
> python3 -c "from src.SleepNode import SleepNode; SleepNode(load=True, name='SleepNode').run()" 
Generating lock file 'dvc.lock'                                                 
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.


In [7]:
!dvc metrics show

Path                           metadata.run:timeit    metadata.sleep_1s:timeitm>
nodes/SleepNode/metadata.json  3.00264                1.00025


We can also time a sinlge function multiple times, using the following example:

In [8]:
@Node(silent=True)
class SleepNodeMulti:
    @TimeIt
    def run(self):
        self.sleep(1)
        self.sleep(2)
    @TimeIt
    def sleep(self, time):
        sleep(time)

In [9]:
SleepNodeMulti()()

In [10]:
!dvc repro

Stage 'SleepNode' didn't change, skipping                             core>
Running stage 'SleepNodeMulti':
> python3 -c "from src.SleepNodeMulti import SleepNodeMulti; SleepNodeMulti(load=True, name='SleepNodeMulti').run()" 
Updating lock file 'dvc.lock'                                                   

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.


In [11]:
!dvc metrics show

Path                                metadata.run:timeit    metadata.sleep:timeit    metadata.sleep_1:timeit    metadata.sleep_1s:timeit
nodes/SleepNode/metadata.json       3.00264                -                        -                          1.00025
nodes/SleepNodeMulti/metadata.json  3.00355                1.001                    2.00202                    -


One can also access the metrics directly within Python. This is possible, because they are just another `zn.metrics` which is automatically added when using one of the given metadata decorators.

In [12]:
SleepNodeMulti(load=True).metadata

{'sleep:timeit': 1.0010027885437012,
 'sleep_1:timeit': 2.0020205974578857,
 'run:timeit': 3.0035529136657715}

In [13]:
temp_dir.cleanup()